In [2]:
import os
import pandas as pd
import numpy as np
import warnings
import datetime
from utils.Preprocessing import label_data, all_phase_sort
from utils.sliding_window import sliding_window
from utils.model import model
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
import tensorflow as tf

In [3]:
#Label the Raw Data (Directory for Labelled = \Processed\Labelled_Data)
label_data('Organized Data')

Processed: Organized Data\brazil\continental_brazil_1648482_statistics.csv
Processed: Organized Data\brazil\electrolux_brazil_1485559_statistics.csv
Processed: Organized Data\canada\hofmannplastics_canada_116239_statistics.csv
Processed: Organized Data\china\abb_china_36405_statistics.csv
Processed: Organized Data\china\abb_china_36435_statistics.csv
Processed: Organized Data\china\abb_china_36440_statistics.csv
Processed: Organized Data\china\abb_china_36445_statistics.csv
Processed: Organized Data\china\abb_china_36450_statistics.csv
Processed: Organized Data\china\abb_china_36455_statistics.csv
Processed: Organized Data\china\abb_china_36460_statistics.csv
Processed: Organized Data\china\abb_china_36475_statistics.csv
Processed: Organized Data\china\abb_china_36480_statistics.csv
Processed: Organized Data\china\abb_china_36497_statistics.csv
Processed: Organized Data\china\abb_china_36501_statistics.csv
Processed: Organized Data\china\abb_china_36505_statistics.csv
Processed: Organi

In [4]:
#Sort Data Containing All phases (Directory containing all phases = \Labelled_Data_Complete_Phase(1-4))
all_phase_sort('Processed\Labelled_Data')

Copied: Processed\Labelled_Data\brazil\continental_brazil_1648482_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\brazil\continental_brazil_1648482_statistics.csv
Copied: Processed\Labelled_Data\brazil\electrolux_brazil_1485559_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\brazil\electrolux_brazil_1485559_statistics.csv
Copied: Processed\Labelled_Data\canada\hofmannplastics_canada_116239_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\canada\hofmannplastics_canada_116239_statistics.csv
Copied: Processed\Labelled_Data\china\abb_china_36405_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\china\abb_china_36405_statistics.csv
Copied: Processed\Labelled_Data\china\abb_china_36435_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\china\abb_china_36435_statistics.csv
Copied: Processed\Labelled_Data\china\abb_china_36440_statistics.csv to Processed/Labelled_Data_Complete_Phase(1-4)\china\abb_china_36440_statistics.csv
Copied

In [6]:
#Loading one mold for training
df=pd.read_csv('Processed/Labelled_Data_Complete_Phase(1-4)/china/abb_china_36475_statistics.csv')
df

,HOUR,MOLD_ID,CT,TAV,SHOT_COUNT,PHASE
0,2020-09-22 16:00:00,36475,0.0,273,0,1
1,2020-09-22 17:00:00,36475,0.0,273,0,1
2,2020-09-22 18:00:00,36475,0.0,276,0,1
3,2020-09-22 19:00:00,36475,0.0,275,0,1
4,2020-09-22 20:00:00,36475,0.0,275,0,1
...,...,...,...,...,...,...
18081,2023-10-29 12:00:00,36475,0.0,225,0,4
18082,2023-10-29 13:00:00,36475,0.0,224,0,4
18083,2023-10-29 14:00:00,36475,0.0,225,0,4
18084,2023-10-29 15:00:00,36475,0.0,221,0,4


In [7]:
#Creating a sliding window 
window_size= 720 #window size is in hours (1D= 24, 1W=168, 1M=720)
x,y = sliding_window(df,window_size)

In [8]:
lstm = model(x,y,'adam')

In [9]:
# Train the model
lstm.fit(x, y, epochs=35, batch_size=12, verbose=2)

Epoch 1/35
1448/1448 - 308s - loss: 0.7925 - accuracy: 0.6880
Epoch 2/35
1448/1448 - 270s - loss: 0.5285 - accuracy: 0.8070
Epoch 3/35
1448/1448 - 267s - loss: 0.4247 - accuracy: 0.8498
Epoch 4/35
1448/1448 - 261s - loss: 0.3844 - accuracy: 0.8625
Epoch 5/35
1448/1448 - 270s - loss: 0.3542 - accuracy: 0.8715
Epoch 6/35


In [ ]:
lstm.save('Weights/abb_china_36505_statistics.h5')

In [ ]:
# Create function to label windowed data
def get_labelled_windows(x, horizon=1):
  """
  Creates labels for windowed dataset.

  E.g. if horizon=1 (default)
  Input: [1, 2, 3, 4, 5, 6] -> Output: ([1, 2, 3, 4, 5], [6])
  """
  return x[:, :-horizon], x[:, -horizon:]

In [ ]:
# Create function to view NumPy arrays as windows 
def make_windows(x, window_size=50, horizon=5):
  """
  Turns a 1D array into a 2D array of sequential windows of window_size.
  """
  # 1. Create a window of specific window_size (add the horizon on the end for later labelling)
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  # print(f"Window step:\n {window_step}")

  # 2. Create a 2D array of multiple window steps (minus 1 to account for 0 indexing)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of size window_size
  # print(f"Window indexes:\n {window_indexes[:3], window_indexes[-3:], window_indexes.shape}")

  # 3. Index on the target array (time series) with 2D array of multiple window steps
  windowed_array = x[window_indexes]

  # 4. Get the labelled windows
  windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

  return windows, labels

In [ ]:
test_window, test_label = get_labelled_windows(tf.expand_dims(tf.range(8)+1, axis=0))
print(f"Window: {tf.squeeze(test_window).numpy()} -> Label: {tf.squeeze(test_label).numpy()}")

In [ ]:
# Apply the function to each window in the DataFrame
for index, row in df.iterrows():
    features = row[['CT', 'TAV', 'SHOT_COUNT','PHASE']].values
    
    # Convert NumPy array to TensorFlow tensor
    features_tensor = tf.convert_to_tensor(features, dtype=tf.float32)
    
    # Assume get_labelled_windows returns windows and labels
    window, label = get_labelled_windows(tf.expand_dims(features_tensor, axis=0))
    
    # Now you can use the window and label as needed
    print(f"Window: {window}, Label: {label}")

In [ ]:
print(window)

In [ ]:
window, label = get_labelled_windows(tf.expand_dims(features, axis=0))

In [ ]:
print(f"Loss: {loss}, Accuracy: {accuracy}")